In [1]:
#importing necessary libraries 
import pandas as pd
import numpy as np
import json
import csv 
import matplotlib.pyplot as plt
import requests
import scipy.stats as st
import os

# importing the csv file
# csvpath = os.path.join("traffic-crashes-1 (2).csv")

# with open(csvpath) as csvfile:

#     csvreader = csv.reader(csvfile, delimiter = ",")

#     csvheader = next(csvreader) 

# importing the csv file 
traffic_data = pd.read_csv("traffic-crashes-1 (2).csv")

traffic_data.columns

Index(['FID_1', 'OBJECTID', 'Crash_Seve', 'County', 'Crash_Date', 'Crash_Year',
       'Crash_Time', 'Primary_St', 'Distance', 'Dir', 'Secondary_', 'Weather',
       'Fatalities', 'Injured', 'Property_D', 'Injury_Typ', 'Crash_Type',
       'Total_Vehi', 'V1_Type', 'V1_Dir', 'V1_Driver_', 'V1_Lane_Nu',
       'V1_Action', 'V1_Driver1', 'V1_Drive_1', 'V1_Vehicle', 'V1_Most_Ha',
       'V1_All_Eve', 'V2_Type', 'V2_Dir', 'V2_Driver_', 'V2_Lane_Nu',
       'V2_Action', 'V2_Driver1', 'V2_Drive_1', 'V2_Vehicle', 'V2_Most_Ha',
       'V2_All_Eve', 'First_Harm', 'Nonmotoris', 'Factors_Ro', 'Lighting',
       'HWY_Factor', 'Agency', 'Accident_R', 'Pedalcycli', 'Pedestrian',
       'Motorcycli', 'AnimalType', 'X', 'Y', 'OBJECTID_1', 'AREA', 'PERIMETER',
       'WARD', 'CNTY_IS', 'SHADE', 'ACRES', 'SQ_MILES', 'AREA_1', 'LEN'],
      dtype='object')

In [2]:
#cleaning the columns, pulling the columns we want to base our analysis off of
clean_traffic_data = traffic_data[["County","Fatalities","Injured","Primary_St","Crash_Seve","Crash_Date","Weather","Crash_Type","V1_Driver1","V1_Driver_","Factors_Ro","Lighting","X","Y"]].copy()

clean_traffic_data.head(5)

,County,Fatalities,Injured,Primary_St,Crash_Seve,Crash_Date,Weather,Crash_Type,V1_Driver1,V1_Driver_,Factors_Ro,Lighting,X,Y
0,CLARK,NaN,0,ECHELON POINT DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,REAR-END,APPARENTLY NORMAL,49,DRY,DAYLIGHT,-115.296856,36.282588
1,CLARK,NaN,1,GRAND CANYON DR,INJURY CRASH,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - NO LIGHTING,-115.305960,36.313917
2,CLARK,NaN,0,SKY POINTE DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - CONTINUOUS LIGHTING,-115.266269,36.279880
3,CLARK,NaN,0,CC215S,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,NaN,0,WET,DARK - SPOT LIGHTING,-115.283121,36.278732
4,CLARK,NaN,0,US95N,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,APPARENTLY NORMAL,27,DRY,DARK - NO LIGHTING,-115.312353,36.327534


In [3]:
# subsituting 0 values for Nan in fatalities 
clean_traffic_data["Fatalities"] = clean_traffic_data["Fatalities"].fillna(0)

# printing the df to double check fillna 
clean_traffic_data.head(5)

,County,Fatalities,Injured,Primary_St,Crash_Seve,Crash_Date,Weather,Crash_Type,V1_Driver1,V1_Driver_,Factors_Ro,Lighting,X,Y
0,CLARK,0.0,0,ECHELON POINT DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,REAR-END,APPARENTLY NORMAL,49,DRY,DAYLIGHT,-115.296856,36.282588
1,CLARK,0.0,1,GRAND CANYON DR,INJURY CRASH,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - NO LIGHTING,-115.305960,36.313917
2,CLARK,0.0,0,SKY POINTE DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - CONTINUOUS LIGHTING,-115.266269,36.279880
3,CLARK,0.0,0,CC215S,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,NaN,0,WET,DARK - SPOT LIGHTING,-115.283121,36.278732
4,CLARK,0.0,0,US95N,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,APPARENTLY NORMAL,27,DRY,DARK - NO LIGHTING,-115.312353,36.327534


In [4]:
# checking columns to see if there is any missing data 
clean_traffic_data.isnull().mean()

County        0.000000
Fatalities    0.000000
Injured       0.000000
Primary_St    0.000000
Crash_Seve    0.000000
Crash_Date    0.000000
Weather       0.000000
Crash_Type    0.000000
V1_Driver1    0.132075
V1_Driver_    0.000000
Factors_Ro    0.106447
Lighting      0.101808
X             0.000000
Y             0.000000
dtype: float64

In [5]:
# double checking the type
type(clean_traffic_data)

pandas.core.frame.DataFrame

In [6]:
# Renaming the columns 
clean_traffic_data = clean_traffic_data.rename(columns={"Crash_Seve": "Crash Severity",
                                                        "Crash_Date": "Crash Date",
                                                        "Crash_Type": "Crash Type",
                                                        "V1_Driver1": "Impairment Level", 
                                                        "V1_Driver_": "Age", 
                                                        "Factors_Ro": "Road Conditions",
                                                        "Primary_St": "Street", 
                                                        "X": "Latitude", 
                                                        "Y": "Longitude"})

clean_traffic_data

,County,Fatalities,Injured,Street,Crash Severity,Crash Date,Weather,Crash Type,Impairment Level,Age,Road Conditions,Lighting,Latitude,Longitude
0,CLARK,0.0,0,ECHELON POINT DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,REAR-END,APPARENTLY NORMAL,49,DRY,DAYLIGHT,-115.296856,36.282588
1,CLARK,0.0,1,GRAND CANYON DR,INJURY CRASH,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - NO LIGHTING,-115.305960,36.313917
2,CLARK,0.0,0,SKY POINTE DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - CONTINUOUS LIGHTING,-115.266269,36.279880
3,CLARK,0.0,0,CC215S,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,NaN,0,WET,DARK - SPOT LIGHTING,-115.283121,36.278732
4,CLARK,0.0,0,US95N,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,APPARENTLY NORMAL,27,DRY,DARK - NO LIGHTING,-115.312353,36.327534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12715,CLARK,0.0,0,N BUFFALO DR,PROPERTY DAMAGE ONLY,12/31/2016,CLOUDY,NON-COLLISION,HAD BEEN DRINKING,24,DRY,DARK - CONTINUOUS LIGHTING,-115.260731,36.203052
12716,CLARK,0.0,1,CLIFF SHADOWS PKWY,INJURY CRASH,12/31/2016,CLEAR,NON-COLLISION,"FELL ASLEEP, FAINTED, FATIGUED, ETC.",68,DRY,DAYLIGHT,-115.329666,36.239294
12717,CLARK,0.0,3,CHEYENNE AVE,INJURY CRASH,12/31/2016,CLOUDY,REAR-END,APPARENTLY NORMAL,65,DRY,DARK - CONTINUOUS LIGHTING,-115.322318,36.218215
12718,CLARK,0.0,2,CRAIG RD,INJURY CRASH,12/31/2016,CLOUDY,ANGLE,HAD BEEN DRINKING,46,DRY,DARK - CONTINUOUS LIGHTING,-115.251390,36.240358


In [7]:
# dropping the rows with nan 
clean_traffic_data = clean_traffic_data.dropna()
clean_traffic_data.head(3)

,County,Fatalities,Injured,Street,Crash Severity,Crash Date,Weather,Crash Type,Impairment Level,Age,Road Conditions,Lighting,Latitude,Longitude
0,CLARK,0.0,0,ECHELON POINT DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,REAR-END,APPARENTLY NORMAL,49,DRY,DAYLIGHT,-115.296856,36.282588
1,CLARK,0.0,1,GRAND CANYON DR,INJURY CRASH,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - NO LIGHTING,-115.305960,36.313917
2,CLARK,0.0,0,SKY POINTE DR,PROPERTY DAMAGE ONLY,01/01/2016,CLEAR,NON-COLLISION,HAD BEEN DRINKING,22,DRY,DARK - CONTINUOUS LIGHTING,-115.266269,36.279880


In [8]:
# double checking new length of data frame
len(clean_traffic_data)

9714

In [9]:
# double checking to see if rows with missing data was dropped
clean_traffic_data.isnull().mean()

County              0.0
Fatalities          0.0
Injured             0.0
Street              0.0
Crash Severity      0.0
Crash Date          0.0
Weather             0.0
Crash Type          0.0
Impairment Level    0.0
Age                 0.0
Road Conditions     0.0
Lighting            0.0
Latitude            0.0
Longitude           0.0
dtype: float64